In [ ]:
import numpy as np


In [ ]:
# import numpy as np
# import os 
# import cv2

# All_data = False

# collected_data_feedback_path = 'data/collected_data_feedback.npy'
# # collected_data_dpo_path = 'data/new_collected_data_dpo.npy'

# ## load data 
# data = np.load(collected_data_feedback_path, allow_pickle=True).item()
# # dpo_data = np.load(collected_data_dpo_path, allow_pickle=True).item()

# print (len(data))

# filtered_data = {}
# max_velocity = 0.2 
# min_velocity = -0.2
# for key, value in data.items():
#     # if not All_data:
#     #     if dpo_data[int(str(key).split('_')[-1])] == 0:
#     #         continue
#     filtered_data[key] = value["tool_speed"] + [value["gripper_angle"]]
#     filtered_data[key][3:6] = [0 for x in filtered_data[key][3:6]]


# for key, value in filtered_data.items():
#     if filtered_data[key][-1] < 0.5:
#         filtered_data[key][-1] = -1
#     else:
#         filtered_data[key][-1] = 1


# episode_length = 80
# ## load image 
# image_folder_path = 'data/pngs'
# ## load files in order
# all_images = {}
# for image_path in sorted(os.listdir(image_folder_path)):
#     image = cv2.imread(os.path.join(image_folder_path, image_path))
#     image_name = image_path.split('.')[0]
#     all_images[image_name] = image

# for index in range(0, int(len(filtered_data)/episode_length)):
#     episode = []
#     for step in range(episode_length):
#         data_name = list(filtered_data.keys())[index + step]
#         episode.append({
#             'image':all_images[data_name],
#             'action': filtered_data[data_name],
#             'language_instruction': 'lift carrot.',
#         })
  
#     # if index < int(0.8 * (len(filtered_data)/episode_length)):
#     #     np.save(f'data/training_data_unorm_100/episode_{index}', episode)
#     # else:
#     #     np.save(f'data/val_data_unorm_100/episode_{index-int(0.8 * (len(filtered_data)/episode_length))}', episode)
  


## Generate Data for Finetuning ##

In [5]:

import numpy as np
import os 
import cv2
import tqdm 
## tfds build --overwrite

collected_data_feedback_path = 'scripted_data_variation/collected_data_feedback.npy'
# collected_data_dpo_path = 'data/new_collected_data_dpo.npy'

data_size = 100 
assert data_size <= 500

## load data 
data = np.load(collected_data_feedback_path, allow_pickle=True).item()

save_dic = {int(i):[] for i in range(0, data_size)}
for key in tqdm.tqdm(data):
    id_ = key
    img_path = f'./scripted_data_variation/pngs/{id_}.png'
    image = cv2.imread(img_path)
    episode_id = id_.split('_')[-1]
    step_id = id_.split('_')[-2]
    tool_speed = data[key]['tool_speed']
    gripper_angle = data[key]['gripper_angle']
    if gripper_angle < 0.5:
        gripper_angle = -1
    else:
        gripper_angle = 1
    action = tool_speed + [gripper_angle]
    action[3:6] = [0 for x in action[3:6]] # optional
    save_dic[int(episode_id)].append({
        'image': image,
        'action': action,
        'language_instruction': 'lift carrot.'
    })
    if int(episode_id) == data_size-1:
        break
    
# create path if not exists
training_path = f'scripted_data_variation/training_data_{data_size}'
val_path = f'scripted_data_variation/val_data_{data_size}'
os.makedirs(training_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)

for episode_key in save_dic:
    if len(save_dic[episode_key]) == 0:
        continue
    if episode_key < int(data_size*0.8):
        np.save(training_path + f'/episode_{episode_key}.npy', save_dic[episode_key])
    else:
        np.save(val_path + f'/episode_{episode_key-int(data_size*0.8)}.npy', save_dic[episode_key])
    # print(f'episode_{episode_key} saved')
    # print(len(save_dic[episode_key]))
print (len(save_dic))

 20%|█▉        | 6126/30989 [00:30<02:01, 203.88it/s]


100


## Generate Data for Motion Commands ##

In [9]:

import numpy as np
import os 
import cv2
import tqdm 
import random
## tfds build --overwrite

def get_random_image(folder_path):
    with os.scandir(folder_path) as entries:
        entries = list(entries)
        random_entry = random.choice(entries)
        if random_entry.is_file():
            image = cv2.imread(random_entry.path)
            return image



collected_data_feedback_path = 'scripted_data_variation/collected_data_feedback.npy'

action_data_size = 500 
assert action_data_size <= 500

image_folder_path = './scripted_data_variation/pngs/'

motion_commands_list = ["Move left.", "Move right.", "Move forward.", "Move backward.", "Move up.", "Move down."]
motion_data_size = 200 * len(motion_commands_list)

## load data 
data = np.load(collected_data_feedback_path, allow_pickle=True).item()

save_dic = {int(i):[] for i in range(0, action_data_size + motion_data_size)}
data_size = action_data_size + motion_data_size
for key in tqdm.tqdm(data):
    id_ = key
    img_path = image_folder_path + f'{id_}.png'
    image = cv2.imread(img_path)
    episode_id = id_.split('_')[-1]
    step_id = id_.split('_')[-2]
    tool_speed = data[key]['tool_speed']
    gripper_angle = data[key]['gripper_angle']
    if gripper_angle < 0.5:
        gripper_angle = -1
    else:
        gripper_angle = 1
    action = tool_speed + [gripper_angle]
    action[3:6] = [0 for x in action[3:6]] # optional
    save_dic[int(episode_id)].append({
        'image': image,
        'action': action,
        'language_instruction': 'lift carrot.'
    })
    if int(episode_id) == action_data_size-1:
        break

## collect data for motion commands only
for index, motion_command in enumerate(motion_commands_list):
    if motion_command == "Move left.":
        action = [0, 0.1, 0, 0, 0, 0, 1]
    elif motion_command == "Move right.":
        action = [0, -0.1, 0, 0, 0, 0, 1]
    elif motion_command == "Move forward.":
        action = [0.1, 0, 0, 0, 0, 0, 1]
    elif motion_command == "Move backward.":
        action = [-0.1, 0, 0, 0, 0, 0, 1]
    elif motion_command == "Move up.":
        action = [0, 0, 0.1, 0, 0, 0, 1]
    elif motion_command == "Move down.":
        action = [0, 0, -0.1, 0, 0, 0, 1]
    for i in tqdm.tqdm(range(200)):
        ## sample a image from image_path_folder
        random_image = get_random_image(image_folder_path)
        save_dic[action_data_size + index * (i+1)].append({
            'image': random_image,
            'action': action,
            'language_instruction': motion_command
        })
    
# create path if not exists
training_path = f'scripted_data_variation/training_data_{action_data_size}'
val_path = f'scripted_data_variation/val_data_{action_data_size}'
os.makedirs(training_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)

for episode_key in save_dic:
    if len(save_dic[episode_key]) == 0:
        continue
    if episode_key < int(data_size*0.8):
        np.save(training_path + f'/episode_{episode_key}.npy', save_dic[episode_key])
    else:
        np.save(val_path + f'/episode_{episode_key-int(data_size*0.8)}.npy', save_dic[episode_key])
    # print(f'episode_{episode_key} saved')
    # print(len(save_dic[episode_key]))
print (len(save_dic))
    

100%|██████████| 200/200 [00:03<00:00, 52.47it/s]


1700


## Generate Data for Random Action with Black Images ##


In [12]:
import numpy as np
import os 
from PIL import Image

min = -0.2 
max = 0.2

step = 0.05 
save_action = []
for x in np.arange(min, max, step):
    for y in np.arange(min, max, step):
        for z in np.arange(min, max, step):
            gripper = 0
            save_action.append([x, y, z, 0, 0, 0, gripper])
            gripper_ = 1
            save_action.append([x, y, z, 0, 0, 0, gripper_])

print (len(save_action))

episode_len = 40
episode_number = len(save_action) // episode_len

def create_balck_image():
    ## create black image with np.uint8
    image_np = np.zeros((480, 640, 3), dtype=np.uint8)
    # save image_np as image type
    image = Image.fromarray(image_np)
    return image

folder_path_training = 'black_data/trianing_data'
folder_path_val = 'black_data/val_data'
os.makedirs(folder_path_training, exist_ok=True)
os.makedirs(folder_path_val, exist_ok=True)

for i in range (0, episode_number):
    episode = []
    for j in range(0, episode_len):
        episode.append({
            'image': create_balck_image(),
            'action': save_action[i*episode_len + j],
            'language_instruction': 'lift carrot.'
        })
    if i < int(0.8 * episode_number):
        np.save(folder_path_training + f'/episode_{i}.npy', episode)
    else:
        np.save(folder_path_val + f'/episode_{i-int(0.8 * episode_number)}.npy', episode)

1024


In [6]:
1240 // 40


31